In [13]:
import os
import shutil
import cv2
import hashlib
from pathlib import Path

from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Project structure (educational style)
PROJECT_ROOT = Path(".")
DATA_DIR = PROJECT_ROOT / "data" / "leapgestrecog"

RAW_DIR = DATA_DIR / "raw"
CLEAN_DIR = DATA_DIR / "clean"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

RAW_DIR.mkdir(parents=True, exist_ok=True)



In [4]:
print("📥 Downloading dataset...")
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Downloaded to:", path)

# Dataset usually contains 'leapGestRecog'
if os.path.isdir(os.path.join(path, "leapGestRecog")):
    src = os.path.join(path, "leapGestRecog")
else:
    src = path

print("Using source:", src)


📥 Downloading dataset...


NameError: name 'kagglehub' is not defined

In [5]:
# Copy raw data once
if not any(RAW_DIR.iterdir()):
    shutil.copytree(src, RAW_DIR, dirs_exist_ok=True)
    print("✅ Raw data copied")

# Create clean data copy
if not CLEAN_DIR.exists():
    shutil.copytree(RAW_DIR, CLEAN_DIR)
    print("✅ Clean data folder created")
else:
    print("ℹ️ Clean data already exists")


NameError: name 'src' is not defined

In [6]:
def remove_corrupted(folder: Path):
    """Remove images that cannot be read."""
    print("\n🧹 Removing corrupted images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            try:
                img = cv2.imread(str(img_path))
                if img is None:
                    img_path.unlink(missing_ok=True)
            except:
                img_path.unlink(missing_ok=True)

In [7]:
def fix_format(folder: Path, size=(224, 224)):
    """Convert images to RGB and resize."""
    print("\n🎨 Standardizing images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize(size)
                img.save(img_path)
            except:
                img_path.unlink(missing_ok=True)

In [8]:
def remove_duplicates(folder: Path):
    """Remove duplicate images using MD5 hash."""
    print("\n🗑️ Removing duplicates...")
    seen = set()
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            with open(img_path, "rb") as f:
                h = hashlib.md5(f.read()).hexdigest()
            if h in seen:
                img_path.unlink(missing_ok=True)
            else:
                seen.add(h)

In [9]:
def denoise(folder: Path):
    """Apply denoising filter."""
    print("\n✨ Denoising images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            try:
                img = cv2.imread(str(img_path))
                img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
                cv2.imwrite(str(img_path), img)
            except:
                pass

In [10]:
def show_balance(folder: Path):
    """Show number of images per class."""
    print("\n📊 Class Balance:")
    for cls in sorted(os.listdir(folder)):
        cls_path = folder / cls
        if cls_path.is_dir():
            print(f"{cls}: {len(os.listdir(cls_path))}")

In [11]:
# Run cleaning pipeline
remove_corrupted(CLEAN_DIR)
fix_format(CLEAN_DIR, IMG_SIZE)
remove_duplicates(CLEAN_DIR)
denoise(CLEAN_DIR)
show_balance(CLEAN_DIR)



🧹 Removing corrupted images...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\leapgestrecog\\clean'

In [12]:
import os
import cv2
import hashlib
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# إعداد المسارات
PROJECT_ROOT = Path(".")
DATA_DIR = PROJECT_ROOT / "data" / "leapgestrecog"
CLEAN_DIR = DATA_DIR / "clean"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# 🧹 إزالة الصور الفاسدة أو غير المدعومة
def remove_corrupted(folder: Path):
    print("\n🧹 Removing corrupted images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                img_path.unlink(missing_ok=True)
                continue
            try:
                img = cv2.imread(str(img_path))
                if img is None:
                    img_path.unlink(missing_ok=True)
            except Exception:
                img_path.unlink(missing_ok=True)

# 🎨 تحويل الصور لـ RGB + إعادة تحجيم
def fix_format(folder: Path, size=(224, 224)):
    print("\n🎨 Standardizing images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize(size)
                img.save(img_path)
            except Exception:
                img_path.unlink(missing_ok=True)

# 🗑️ إزالة الصور المكررة باستخدام MD5
def remove_duplicates(folder: Path):
    print("\n🗑️ Removing duplicates...")
    seen = set()
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            try:
                with open(img_path, "rb") as f:
                    h = hashlib.md5(f.read()).hexdigest()
                if h in seen:
                    img_path.unlink(missing_ok=True)
                else:
                    seen.add(h)
            except Exception:
                img_path.unlink(missing_ok=True)

# ✨ إزالة الضوضاء من الصور
def denoise(folder: Path):
    print("\n✨ Denoising images...")
    for cls in tqdm(os.listdir(folder)):
        cls_path = folder / cls
        if not cls_path.is_dir():
            continue
        for file in os.listdir(cls_path):
            img_path = cls_path / file
            if img_path.is_dir():
                continue
            try:
                img = cv2.imread(str(img_path))
                if img is not None:
                    img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
                    cv2.imwrite(str(img_path), img)
            except Exception:
                pass

# 📊 عرض عدد الصور في كل فئة + تحذير لو فيه عدم توازن
def show_balance(folder: Path):
    print("\n📊 Class Balance:")
    counts = {}
    for cls in sorted(os.listdir(folder)):
        cls_path = folder / cls
        if cls_path.is_dir():
            counts[cls] = len(os.listdir(cls_path))
            print(f"{cls}: {counts[cls]}")
    avg = sum(counts.values()) / len(counts)
    for cls, count in counts.items():
        if count < 0.5 * avg:
            print(f"⚠️ Warning: Class '{cls}' has fewer samples ({count}) than average ({avg:.0f})")

# 🚀 تشغيل الـ pipeline كله
remove_corrupted(CLEAN_DIR)
fix_format(CLEAN_DIR, IMG_SIZE)
remove_duplicates(CLEAN_DIR)
denoise(CLEAN_DIR)
show_balance(CLEAN_DIR)

# 🔥 إعداد الـ datagen مع augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = train_datagen.flow_from_directory(
    CLEAN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="training",
    class_mode="categorical"
)

val_gen = train_datagen.flow_from_directory(
    CLEAN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="validation",
    class_mode="categorical"
)

# 🧠 بناء EfficientNetB0 مع transfer learning
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False  # نجمّد الـ feature extractor في البداية

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=preds)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# تدريب أولي
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# Fine-tuning: فك تجميد بعض الطبقات
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# حفظ النموذج
model.save("efficientnet_gesture.h5")

ModuleNotFoundError: No module named 'tensorflow'